In [ ]:
# %%

from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.tabular.configs.hyperparameter_configs import get_hyperparameter_config

import numpy as np

from sklearnex import patch_sklearn
patch_sklearn()

label = 'signature'

data_url = 'https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/'

train_data = TabularDataset(f'{data_url}train.csv')
test_data = TabularDataset(f'{data_url}test.csv')

train_data.head()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


,Unnamed: 0,chern_simons,cusp_volume,hyperbolic_adjoint_torsion_degree,hyperbolic_torsion_degree,injectivity_radius,longitudinal_translation,meridinal_translation_imag,meridinal_translation_real,short_geodesic_imag_part,short_geodesic_real_part,Symmetry_0,Symmetry_D3,Symmetry_D4,Symmetry_D6,Symmetry_D8,Symmetry_Z/2 + Z/2,volume,signature
0,70746,0.090530,12.226322,0,10,0.507756,10.685555,1.144192,-0.519157,-2.760601,1.015512,0.0,0.0,0.0,0.0,0.0,1.0,11.393225,-2
1,240827,0.232453,13.800773,0,14,0.413645,10.453156,1.320249,-0.158522,-3.013258,0.827289,0.0,0.0,0.0,0.0,0.0,1.0,12.742782,0
2,155659,-0.144099,14.761030,0,14,0.436928,13.405199,1.101142,0.768894,2.233106,0.873856,0.0,0.0,0.0,0.0,0.0,0.0,15.236505,2
3,239963,-0.171668,13.738019,0,22,0.249481,27.819496,0.493827,-1.188718,-2.042771,0.498961,0.0,0.0,0.0,0.0,0.0,0.0,17.279890,-8
4,90504,0.235188,15.896359,0,10,0.389329,15.330971,1.036879,0.722828,-3.056138,0.778658,0.0,0.0,0.0,0.0,0.0,0.0,16.749298,4


In [ ]:
# %%

custom_hyperparameters = get_hyperparameter_config('zeroshot')
custom_hyperparameters['NN_TORCH'] = [{}]
custom_hyperparameters['CAT'] = [{}]
custom_hyperparameters['RF'] = [{'criterion': 'log_loss', 'n_estimators':5000, 'ag_args': {'name_suffix': 'LogLoss'}}]
custom_hyperparameters['XT'] = [{'criterion': 'log_loss', 'n_estimators':5000, 'ag_args': {'name_suffix': 'LogLoss'}}]

custom_hyperparameters['LR'] = [
    {'multi_class':'multinomial', 'penalty':None, 'tol':1e-6, 'max_iter':10000,
        'ag_args': {'name_suffix': 'Base'}},
    {'multi_class':'multinomial', 'penalty':'l2', 'tol':1e-6, 'max_iter':10000, 'C':0.1,
        'ag_args': {'name_suffix': 'Ridge'}},
    {'multi_class':'multinomial', 'penalty':'l1', 'tol':1e-6, 'max_iter':10000, 'C':0.1,
        'ag_args': {'name_suffix': 'Lasso'}},
    {'multi_class':'multinomial', 'penalty':'elasticnet', 'tol':1e-6, 'max_iter':10000, 'C':0.1,
        'ag_args': {'name_suffix': 'ElasticNet'}}
    ]

custom_preset = {'auto_stack': False, 'dynamic_stacking': False,
                'hyperparameters':custom_hyperparameters, 'refit_full': False,
                'set_best_to_refit_full': False, 'save_bag_folds': False}

In [ ]:
# %%

np.random.seed(2024)

predictor = TabularPredictor(label=label, problem_type='multiclass', eval_metric='log_loss', log_to_file=True)

predictor.fit(train_data, presets=custom_preset, excluded_model_types=['KNN'])  

No path specified. Models will be saved in: "AutogluonModels\ag-20240818_023033"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22635
CPU Count:          8
Memory Avail:       8.60 GB / 15.70 GB (54.8%)
Disk Space Avail:   751.23 GB / 952.43 GB (78.9%)
Presets specified: [{'auto_stack': False, 'dynamic_stacking': False, 'hyperparameters': {'NN_TORCH': [{}], 'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, 'GBMLarge', {'extra_trees': False, 'feature_fraction': 0.7023601671276614, 'learning_rate': 0.012144796373999013, 'min_data_in_leaf': 14, 'num_leaves': 53, 'ag_args': {'name_suffix': '_r131', 'priority': -3}}, {'extra_trees': True, 'feature_fraction': 0.5636931414546802, 'learning_rate': 0.01518660230385841, 'min_data_in_leaf': 48, 'num_leaves': 16, 'ag_args': {'name_suffix': '_r96', 'priority': -6

[1000]	valid_set's multi_logloss: 0.220428
[2000]	valid_set's multi_logloss: 0.183969
[3000]	valid_set's multi_logloss: 0.177531
[4000]	valid_set's multi_logloss: 0.183352


	-0.1772	 = Validation score   (-log_loss)
	29.59s	 = Training   runtime
	1.27s	 = Validation runtime
Fitting model: XGBoost_r33 ...
	-0.1502	 = Validation score   (-log_loss)
	20.7s	 = Training   runtime
	0.29s	 = Validation runtime
Fitting model: NeuralNetFastAI_r102 ...
	-0.182	 = Validation score   (-log_loss)
	3.65s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBM_r188 ...
	-0.2136	 = Validation score   (-log_loss)
	14.53s	 = Training   runtime
	0.25s	 = Validation runtime
Fitting model: NeuralNetFastAI_r145 ...
	-0.1538	 = Validation score   (-log_loss)
	35.35s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: XGBoost_r89 ...
	-0.1499	 = Validation score   (-log_loss)
	8.46s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: LightGBM_r130 ...
	-0.1581	 = Validation score   (-log_loss)
	9.01s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: NeuralNetFastAI_r11 ...
	-0.2761	 = Validation score   (-log_loss)
	

[1000]	valid_set's multi_logloss: 0.305756
[2000]	valid_set's multi_logloss: 0.22039
[3000]	valid_set's multi_logloss: 0.205861
[4000]	valid_set's multi_logloss: 0.208957


	-0.205	 = Validation score   (-log_loss)
	118.24s	 = Training   runtime
	2.65s	 = Validation runtime
Fitting model: NeuralNetFastAI_r95 ...
	-0.1487	 = Validation score   (-log_loss)
	37.54s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: XGBoost_r98 ...
	-0.1591	 = Validation score   (-log_loss)
	51.57s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: LightGBM_r15 ...
	-0.1563	 = Validation score   (-log_loss)
	7.5s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: NeuralNetFastAI_r37 ...
	-0.1686	 = Validation score   (-log_loss)
	15.66s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBM_r143 ...
	-0.1514	 = Validation score   (-log_loss)
	24.06s	 = Training   runtime
	0.35s	 = Validation runtime
Fitting model: NeuralNetFastAI_r134 ...
	-0.1755	 = Validation score   (-log_loss)
	14.35s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: LightGBM_r94 ...


[1000]	valid_set's multi_logloss: 0.187907


	-0.1863	 = Validation score   (-log_loss)
	9.34s	 = Training   runtime
	0.44s	 = Validation runtime
Fitting model: NeuralNetFastAI_r111 ...
	-0.2386	 = Validation score   (-log_loss)
	3.46s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: NeuralNetFastAI_r65 ...
	-0.4668	 = Validation score   (-log_loss)
	6.32s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: NeuralNetFastAI_r88 ...
	-0.1782	 = Validation score   (-log_loss)
	7.74s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBM_r30 ...


[1000]	valid_set's multi_logloss: 0.179381


	-0.1787	 = Validation score   (-log_loss)
	30.01s	 = Training   runtime
	0.84s	 = Validation runtime
Fitting model: XGBoost_r49 ...
	-0.1442	 = Validation score   (-log_loss)
	10.27s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: XGBoost_r31 ...
	-0.1555	 = Validation score   (-log_loss)
	62.98s	 = Training   runtime
	0.95s	 = Validation runtime
Fitting model: NeuralNetFastAI_r160 ...
	-0.2795	 = Validation score   (-log_loss)
	22.55s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: LightGBM_r135 ...
	-0.1519	 = Validation score   (-log_loss)
	14.31s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: XGBoost_r22 ...
	-0.1525	 = Validation score   (-log_loss)
	7.57s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: NeuralNetFastAI_r69 ...
	-0.1827	 = Validation score   (-log_loss)
	20.84s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: NeuralNetFastAI_r138 ...
	-0.1579	 = Validation score   (-log_loss)


In [ ]:
# %%

metrics = ['model', 'score_test', 'score_val', 'eval_metric', 'pred_time_test', 'fit_time']

df_leaders = predictor.leaderboard(test_data)

df_leaders.head(40)[metrics]

,model,score_test,score_val,eval_metric,pred_time_test,fit_time
0,WeightedEnsemble_L2,-0.135689,-0.117625,log_loss,4.895841,205.494122
1,NeuralNetFastAI_r145,-0.157206,-0.153805,log_loss,0.250277,35.354512
2,NeuralNetFastAI_r191,-0.157977,-0.146975,log_loss,0.196200,32.869906
3,CatBoost,-0.158654,-0.150310,log_loss,1.222955,222.880075
4,NeuralNetFastAI_r95,-0.160505,-0.148729,log_loss,0.278248,37.544935
5,XGBoost_r49,-0.164594,-0.144229,log_loss,0.883828,10.266812
6,NeuralNetFastAI_r138,-0.165084,-0.157938,log_loss,0.280429,63.767979
7,XGBoost_r34,-0.165541,-0.155691,log_loss,1.212304,17.105341
8,NeuralNetFastAI_r103,-0.166382,-0.149602,log_loss,0.167598,25.833703
9,XGBoost_r33,-0.167556,-0.150230,log_loss,1.931661,20.701121


In [ ]:
# %%

df_leaders.tail(20)[metrics]

,model,score_test,score_val,eval_metric,pred_time_test,fit_time
38,LightGBM_r96,-0.205891,-0.177163,log_loss,8.032906,29.594550
39,NeuralNetFastAI_r172,-0.207842,-0.187473,log_loss,0.092462,9.516577
40,LightGBM_r94,-0.210784,-0.186272,log_loss,2.827439,9.336618
41,LightGBM_r196,-0.214684,-0.204984,log_loss,18.787951,118.236710
42,LightGBM_r188,-0.218027,-0.213627,log_loss,1.911377,14.533842
43,LightGBM_r42,-0.226001,-0.216432,log_loss,1.225813,9.813632
44,NeuralNetFastAI_r4,-0.228713,-0.223634,log_loss,0.126091,18.282894
45,NeuralNetFastAI_r111,-0.248465,-0.238556,log_loss,0.082183,3.460618
46,ExtraTreesLogLoss,-0.259185,-0.265808,log_loss,4.102149,11.888993
47,NeuralNetFastAI_r160,-0.261677,-0.279471,log_loss,0.291072,22.551296


In [ ]:
# %%

y_pred = predictor.predict_proba(test_data.drop(columns=[label]))
y_pred.head()

# predictor.evaluate(test_data, silent=True)

,-12,-10,-8,-6,-4,-2,0,2,4,6,8,10,12
0,7.340542e-06,3.625383e-06,7.729577e-06,0.000057,0.999690,0.000108,0.000054,0.000034,0.000023,0.000009,3.484007e-06,1.194217e-06,1.612354e-06
1,1.612201e-05,1.095125e-05,2.221138e-05,0.000095,0.000417,0.533979,0.464041,0.001008,0.000271,0.000094,2.274319e-05,1.059211e-05,1.169404e-05
2,1.183407e-06,5.222307e-07,8.397458e-07,0.000003,0.000008,0.000212,0.999736,0.000025,0.000008,0.000003,8.293446e-07,5.942641e-07,6.764009e-07
3,7.488665e-07,5.697032e-07,1.052041e-06,0.000016,0.000025,0.000019,0.000025,0.000073,0.999033,0.000801,2.992499e-06,5.714320e-07,1.338271e-06
4,9.010996e-07,6.757476e-07,1.255323e-06,0.000005,0.000014,0.000029,0.012253,0.987516,0.000173,0.000005,1.050195e-06,5.764222e-07,8.332386e-07
